In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from skimage.transform import resize

# Prepare Data

In [ ]:
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train.head()

from train datafram, we might use Id in for loop to extract image

## Extract and resize image

I decided to resize all image into 128 x 128 pixel

In [ ]:
all_image=[]
count=0
for index in range(train.shape[0]):
    id=train.loc[index,'Id']
    path='../input/petfinder-pawpularity-score/train/'+str(id)+'.jpg'
    im_array=cv2.imread(path)
    im_array=resize(im_array,(128,128),anti_aliasing=True)
    all_image.append(im_array)
    
    count+=1
    progress=(count/len(train['Id']))*100
    print('progress =',round(progress,2), '%',end='\r')

In [ ]:
X=np.array(all_image)
X=X.reshape(-1,128,128,3)

del all_image
X.shape

In [ ]:
y=train['Pawpularity']
y.shape

# Neural Network

I adapt neural network structure from this [notebook](https://www.kaggle.com/guidosalimbeni/regression-with-convolutional-neural-network-keras#Build-and-train-the-CNN-model)

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(128,128,3),padding='same'))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='linear'))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

In [ ]:
model.fit(X,y,epochs=50)

# Predict pet pawpularity

In [ ]:
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test.head()

In [ ]:
all_image=[]
for index in range(test.shape[0]):
    id=test.loc[index,'Id']
    path='../input/petfinder-pawpularity-score/test/'+str(id)+'.jpg'
    im_array=cv2.imread(path)
    im_array=resize(im_array,(128,128),anti_aliasing=True)
    all_image.append(im_array)

X_test=np.array(all_image).reshape(-1,128,128,3)
del all_image

y_predict=model.predict(X_test)
y_predict=y_predict.reshape(-1,)

submission_df=pd.DataFrame({'Id':test['Id'],'Pawpularity':y_predict})
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)